In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
from torch.utils.data import DataLoader,Dataset
import torch.utils.data as data
from torchvision import transforms
import torchvision

from torchinfo import summary


import sys,os
sys.path.append(os.pardir)

import py_func.Deep_learning_torch_func as DL
import py_func.models_func as my_model
import py_func.dat_rb_func as drb
import py_func.my_numpy_class as mnc
import py_func.my_torch_class as mtc
import py_func.models_func as my_model

import py_func.Deep_learning_torch_func as DL
import py_func.dat_rb_func as drb

from torch.optim import lr_scheduler
import numpy as np
import importlib
import os.path as osp


import matplotlib.pyplot as plt

### FashionMNIST experiment dataset 68000train shuffle

In [3]:
outputdir = "../output/experiment/SLM_phase/fm"
howdatadir = osp.join(outputdir, "id/midline_randomshuffle_68000train/")

datapath = '../dat/experiment/SLM/fm_6_18/fm_1_70000_slt0.4_0.8_N3_lsd_midline.dat'
labelpath='../dat/label/fm_label.dat'

shfldir= "../output/experiment/SLM_phase"
shflidpath=osp.join(shfldir, "shuffle_first_68000.npy")
trainid = np.load(file=shflidpath)
shflidpath=osp.join(shfldir, "shuffle_68001_69000.npy")
valid = np.load(file=shflidpath)
shflidpath=osp.join(shfldir, "shuffle_69001_70000.npy")
testid = np.load(file=shflidpath)



fm = mnc.My_numpy(1, datapath)
fm.simread(70000, 1600)
print(fm.data.shape)

fm.data = fm.data.reshape(70000, 1, 1600)
print(fm.data.shape)

fmlbl = mnc.My_numpy(4, labelpath)
fmlbl.labelread(70000)
print(fmlbl.data.shape)


myset = mtc.My_dataset2(fm.data, fmlbl.data)
myset.numpy2tensor_labelint()
traindata, trainlabel = myset.index_splitdata(trainid)
valdata, vallabel = myset.index_splitdata(valid)
testdata, testlabel = myset.index_splitdata(testid)

myset.set3data(traindata, val=valdata, test=testdata)
myset.set3label(trainlabel, val=vallabel, test=testlabel)

myset.datanormalize()
myset.tensor2dataset()

batch=256
mlt = DataLoader(myset.dataset_train, batch_size = batch, shuffle = True)
mlv = DataLoader(myset.dataset_val, batch_size = batch, shuffle = False)
mltest = DataLoader(myset.dataset_test, batch_size = batch, shuffle = False)

print(len(mlt), len(mlv), len(mltest))

(70000, 1600)
(70000, 1, 1600)
(70000,)
266 4 4


#### model notsave paramater search

In [4]:
testnum=40
alphamax=5     #0<=alpha=<alphamax
stddevmax=0.05 #0<=stddev=<stddevmax

alphard=np.random.rand(testnum)*alphamax
stddevrd=np.random.rand(testnum)*stddevmax

In [5]:
alphard

array([4.84108561e+00, 1.68708204e+00, 4.10937201e+00, 2.21774842e+00,
       1.62733747e+00, 2.64207549e+00, 1.68145226e+00, 7.67553075e-02,
       1.71538354e+00, 3.22619050e-03, 3.97480007e-02, 1.88304041e+00,
       4.87589445e+00, 3.06995795e-01, 4.95654916e+00, 1.37839603e-01,
       2.26071170e+00, 3.98057624e+00, 4.74241807e+00, 1.01383949e+00,
       2.58486423e+00, 1.85638325e+00, 3.19909428e+00, 1.37414903e+00,
       1.84177731e+00, 4.37710853e+00, 1.53496996e+00, 2.46577231e+00,
       4.59863911e+00, 2.04931337e+00, 3.60533381e+00, 4.24619354e+00,
       2.81064863e+00, 2.75539300e+00, 2.68398022e+00, 2.25110117e+00,
       9.78669238e-01, 4.68760074e+00, 4.14670902e+00, 4.79095414e+00])

In [6]:
stddevrd

array([0.02260023, 0.02737619, 0.04701132, 0.00792452, 0.04280641,
       0.01778611, 0.04396987, 0.04417447, 0.02205042, 0.04234056,
       0.02976362, 0.04971071, 0.00098905, 0.02032555, 0.02421627,
       0.02653352, 0.04282425, 0.04363615, 0.0191795 , 0.04068455,
       0.03725795, 0.04397387, 0.0448444 , 0.03723211, 0.00513029,
       0.01517971, 0.00797895, 0.01310297, 0.03494945, 0.01916647,
       0.03135016, 0.00410244, 0.03669398, 0.01834877, 0.0005486 ,
       0.00730747, 0.04842167, 0.03285981, 0.00229354, 0.02941858])